In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os
import re

load_dotenv()

client = OpenAI(
    api_key=os.getenv('GROQ_TOKEN'),
    base_url=os.getenv('BASE_URL_GROQ')
)

def get_response(prompt: str) -> str:
    response = client.chat.completions.create(
        model=os.getenv('GROQ_MODEL'),
        messages=[
            {'role':'user','content':prompt}
        ]
    )
    return response.choices[0].message.content

def clean_response(text):
    # Паттерн ищет всё между <think> и </think>, включая переносы строк
    pattern = r"<think>.*?</think>"
    
    # Заменяем найденное на пустоту и убираем лишние пробелы по краям
    cleaned_text = re.sub(pattern, "", text, flags=re.DOTALL).strip()
    return cleaned_text

In [2]:
## Часть 1
### Задача 1.1
product_reviews = """
Отзыв 1 (Иван П., 15.02.2024):
Купил новый смартфон TechPhone X5 две недели назад. Камера действительно впечатляет -
фотографии получаются четкими даже при слабом освещении. Батарея держит весь день при
активном использовании. Однако экран слишком яркий даже на минимальных настройках, что
напрягает глаза. Также разочаровала медленная зарядка - полная зарядка занимает почти
3 часа. За эту цену ожидал большего.

Отзыв 2 (Мария С., 18.02.2024):
Отличный телефон! Быстрый, стильный, экран яркий и четкий. Особенно радует, что он
поддерживает все современные приложения без зависаний. Немного тяжеловат, но это не
критично. Рекомендую всем, кто ищет надежный смартфон в среднем ценовом сегменте.

Отзыв 3 (Алексей Н., 20.02.2024):
После месяца использования могу сказать, что телефон хороший, но не без недостатков.
Производительность на высоте, камера отличная, звук чистый. Но есть проблема с перегревом
при играх - телефон становится горячим через 20-30 минут. Служба поддержки сказала, что
это нормально, но меня это беспокоит.

Отзыв 4 (Елена К., 22.02.2024):
Ужасный опыт покупки! Заказала телефон онлайн, доставка задержалась на неделю без
объяснений. Когда наконец получила, обнаружила царапину на задней панели. Связалась с
поддержкой - долго не отвечали, потом предложили скидку 5% на следующую покупку вместо
замены. Сам телефон работает нормально, но сервис просто отвратительный.
"""

basic_prompt = f'''Резюмируй отзывы обрамленные тройными обратными кавычками в три предложения.

```{product_reviews}```'''

focus_prompt = f'''Резюмируй отзывы обрамленные тройными обратными кавычками. Извлеки только технические характеристики продукта в формате маркированного списка

```{product_reviews}```'''

structed_prompt = f'''Резюмируй отзывы обрамленные тройными обратными кавычками. Резюме должно быть сформированно в следующем формате:
- Позитивные аспекты (что хвалят)
- Негативные аспекты (на что жалуются, что критикуют)
- Проблемы с сервисом
- Общая рекомендация для производителя не более 3 предложений

```{product_reviews}```'''

prompt_dict = {
    'БАЗОВОЕ РЕЗЮМЕ':basic_prompt,
    "ТЕХНИЧЕСКИЕ ХАРАКЕТРИСТИКИ":focus_prompt,
    "СТРУКТУРИРОВАННЫЙ АНАЛИЗ":structed_prompt
}

for i, prompt_name in enumerate(prompt_dict):
    print(f'=== {prompt_name} ===')
    print(clean_response(get_response(prompt_dict[prompt_name])))

=== БАЗОВОЕ РЕЗЮМЕ ===
Отличная камера, высокая производительность и качественный экран отметили пользователи, однако экран слишком яркий, зарядка медленная, а в процессе игр возможен перегрев. Проблемы с обслуживанием и небольшой вес стали сдерживающим фактором для некоторых. Несмотря на достоинства, критика касается сервиса и отдельных технических недостатков.
=== ТЕХНИЧЕСКИЕ ХАРАКЕТРИСТИКИ ===
Маркированный список технических характеристик:  
- **Камера**  
  - Высокое качество съемки в условиях слабого освещения (Иван П., 15.02.2024)  
- **Батарея**  
  - Долгий срок жизни батареи при активном использовании (Иван П., 15.02.2024)  
- **Экран**  
  - Высокая яркость, вызывающая неудобства на минимальных настройках (Иван П., 15.02.2024)  
  - Яркий и четкий дисплей (Мария С., 18.02.2024)  
- **Производительность**  
  - Мощный процессор, поддерживающий современные приложения без задержек (Мария С., 18.02.2024)  
  - Отличная производительность при обычных задачах (Алексей Н., 20.02.20

In [3]:
### Задача 1.2
versions = {
    'audience': ['Технически подкованные пользователи', "Семейная аудитория", "Бизнес-сегмент"],
    'words_limit': ['150-200 слов', "120-150 слов", "100-120 слов"],
    'tone': ['Профессиональный тон', "Дружелюбный тон", "Формальный тон"]
}

for i in range(3):
    prompt = f'''Сгенерируй маркетинговый текст, по следующим правилам:
    - Заголовок должен привлекать внимание аудитории
    - Основное описание с акцентов на потребности аудитории
    - Текст должен содержать призыв к действию (CTA)

    Параметры:
    - Целевая аудитория: {versions['audience'][i]}
    - Лимит текста: {versions['words_limit'][i]}
    - Тональность текста: {versions['tone'][i]}
    '''

    print(f'=== {versions['audience'][i].upper()} ===')
    print(clean_response(get_response(prompt)))

=== ТЕХНИЧЕСКИ ПОДКОВАННЫЕ ПОЛЬЗОВАТЕЛИ ===
**Заголовок:**  
**Превращайте Ваши Системы В Ракеты: Инновационные Инструменты для Технических Экспертов**  

**Основное описание:**  
Если вы разрабатываете, управляете или повышаете производительность сложных систем, вам знакомы вызовы: сбои сервисов, узкие места в производительности, сложные интеграции или неэффективная обработка данных. Наши решения разработаны, чтобы решать эти проблемы – быстро, надежно и масштабируемо.  

Наши платформы и инструменты сочетают в себе мгновенная обработка данных, отказоустойчивую архитектуру и гибкую модульность, позволяя интегрироваться с существующими решениями без потери скорости. Повысьте устойчивость к атакам с помощью многуровневой защиты, оптимизируйте ресурсы с интеллектуальными алгоритмами и минимизируйте время на техническое обслуживание благодаря автоматизации. Для команд, ценящих производительность и точность, мы предлагаем инструменты, которые «протестированы в реальных условиях» — тысячами

In [4]:
## Часть 2
### Задача 2.1
informal_message = """
Привет команда!!!
слушайте вчера обсуждали с Петровым из отдела продаж насчет нового проекта
типа надо бы запустить маркетинговую кампанию для нового продукта уже к концу месяца
бюджет у нас около 500к
кто будет заниматься дизайном баннеров и лендинга??
и еще нужно подумать про таргетинг в соцсетях
давайте встретимся в четверг обсудим детали
"""

print('=== ШАГ 1 ===')
prompt_1 = f"""Исправь грамматику текста обрамленного торойными обратными кавычками 
```{informal_message}```"""

response_1 = clean_response(get_response(prompt_1))
print(response_1)
print('=== ШАГ 2 ===')
prompt_2 = f"""Изменить тональность у текста обрамленного тройными обратными кавычками на формальный деловой стиль
```{response_1}```"""

response_2 = clean_response(get_response(prompt_2))
print(response_2)
print('=== ШАГ 3 ===')
prompt_3 = f"""Структурируй текст обрамленный тройными обратными кавычками в формат email:
- Укажи тему
- Добавь блок приветствия
- Добавь блок основной части
- Добавь блок призыв к действию
- Добавь блок подписи
```{response_2}```"""

response_3 = clean_response(get_response(prompt_3))
print(response_3)
print('=== ШАГ 4 ===')
prompt_4 = f"""Добавь к тексту письма обрамленного тройными обратными кавычками:
- Точные сроки
- Зоны ответственности
- Дальнейшие шаги
```{response_3}```"""

response_4 = clean_response(get_response(prompt_4))
print(response_4)

=== ШАГ 1 ===
Вот исправленный и оптимизированный вариант текста:

---  
Привет, команда!  
Слышите: вчера с Петровым из отдела продаж обсуждали новый проект — нужно запустить маркетинговую кампанию для нового продукта к концу месяца. Бюджет: ~500 000 рублей.  

Вопросы:  
1. Кто будет делать дизайн баннеров и landing-page'ов?  
2. Нужно обсудить таргетинг в социальных сетях.  

Давайте свернёмся в четверг и пройдём детали.  
---  

**Пояснения по редактуре:**  
1. Разбита длинная фраза на части для структурированности.  
2. Использованы эмодзи-символы (❗️, ❓) для акцентирования приоритетов.  
3. Добавлены маркировка и абзацы для улучшения читаемости.  
4. Исправлены разговорные выражения ("типа", "всё бы") на более корректные формулировки.  
5. Уточнены цифровые значения ("500 000 рублей" вместо "500к").  
6. Сокращён уровень неразговорности ("соцсетях" → "социальных сетях").  

Текст стал структурированным и проще для восприятия, сохранив прагматику командного коммуникационного стиля

In [5]:
### Задача 2.2
technical_text = """
Наша платформа использует RESTful API архитектуру для взаимодействия с клиентскими
приложениями. Аутентификация реализована через OAuth 2.0 протокол с использованием
JWT токенов. Данные передаются в JSON формате через HTTPS соединение. Система
поддерживает горизонтальное масштабирование через Kubernetes оркестрацию контейнеров
Docker. Для обеспечения высокой доступности используется архитектура микросервисов с
репликацией базы данных PostgreSQL и кэшированием через Redis.
"""

audiences = {
    "ТОП МЕНЕДЖМЕНТ":'''Для топ-менеджмента (нетехническая аудитория) со следующими условиями для адаптации текст:
    - Простым языком без технических терминов
    - С фокусировкой на бизнес-ценности
    - Максимум 3 предложения
    ''',
    "КЛИЕНТЫ":"""Для клиентов которые понимают базовые основы технологий со следующими условиями для адаптации текста:
    - Понятным языком с минимумом технических терминов
    - С фокусировкой на преимущества для пользователя
    - Максимум 4 предложения""",
    "ТЕХ ДОКУМЕНТАЦИЯ":"""Для технической документации которую читает техническая аудитория со следующими условиями для адаптации:
    - С сохранением технической терминологии
    - С добавлением большего колличества деталей
    - Структурируй текст на компоненты
    - Максимум 5 предложений"""
}

for audience in audiences:
    print(f'=== {audience} ===')
    prompt = f'''Адаптируй технический текст обрамленный тройными обратными кавычками {audiences[audience]}

    ```{technical_text}```'''
    print(clean_response(get_response(prompt)))

=== ТОП МЕНЕДЖМЕНТ ===
Наша платформа обеспечивает безопасное и устойчивое взаимодействие с приложениями, используя современные технологии защиты данных и шифрования. Она легко масштабируется, чтобы справляться с ростом нагрузки, и не теряет производительность даже при высоком потоке запросов. Это гарантирует бесперебойную работу сервисов и оперативную обработку данных для ваших пользователей.
=== КЛИЕНТЫ ===
Наша система обеспечивает простое и безопасное подключение к приложениям, используя стандартные форматы данных. Пользователей защищает современная система входа без хранения паролей, а данные передаются по защищенному каналу. Благодаря гибкой архитектуре система не прерывает работу даже при увеличении пользователей. Мы сохраняем высокую надёжность данных и скорость откликов благодаря продвинутому кэшированию и резервированию информации.
=== ТЕХ ДОКУМЕНТАЦИЯ ===
```
Наши RESTful API реализуют набор эндпоинтов с версионностью v1.0, поддерживающих методы GET, POST, PUT, DELETE, с мар

In [6]:
## Часть 3
### Задача 3.1

support_tickets = [
    """Тикет #1001
    От: client1@email.com
    Дата: 25.02.2024

    Не могу войти в личный кабинет третий день. Пробовал сбросить пароль через
    форму восстановления, но письмо не приходит ни на основную, ни на резервную
    почту. Очень срочно - нужно оплатить счет до конца недели!""",

    """Тикет #1002
    От: company@business.com
    Дата: 25.02.2024

    Добрый день. Мы крупный клиент вашей компании (контракт на 5 млн руб/год).
    Хотели бы обсудить возможность интеграции вашего API с нашей CRM системой.
    Также интересуют корпоративные скидки для увеличения объема услуг.""",

    """Тикет #1003
    От: user55@mail.ru
    Дата: 26.02.2024

    В мобильном приложении есть небольшая проблема: когда поворачиваешь экран в
    горизонтальное положение на странице статистики, часть графиков обрезается.
    Не критично, но немного неудобно.""",

    """Тикет #1004
    От: angry_customer@gmail.com
    Дата: 26.02.2024

    ЭТО ВОЗМУТИТЕЛЬНО!!! Вы списали деньги с моей карты за подписку, которую я
    отменил ЕЩЕ ДВА МЕСЯЦА НАЗАД! Требую немедленного возврата средств и компенсации!
    Уже написал жалобу в Роспотребнадзор. Если не решите в течение 24 часов,
    обращаюсь в суд!""",

    """Тикет #1005
    От: developer@startup.com
    Дата: 27.02.2024

    Обнаружил критическую уязвимость в вашем API - возможна SQL инъекция через
    параметр user_id в endpoint /api/v2/users. Детали отправил на security@yourcompany.com.
    Рекомендую срочно пропатчить.""",

    """Тикет #1006
    От: newuser@domain.com
    Дата: 27.02.2024

    Здравствуйте! Только начал пользоваться вашим сервисом. Подскажите, пожалуйста,
    как настроить уведомления на email? В документации не нашел этот раздел."""
]

classifications = []

print('=== КЛАССИФИКАЦИЯ ТИКЕТОВ ===')
for ticket in support_tickets:
    prompt = f'''Классифицируй тикет обрамленный тройными обратными кавычками по трем параметрам:
    1. **Категория**:
    - Технические проблемы
    - Вопросы об оплате/возврате
    - Запрос информации
    - Бизнес-предложение
    - Жалоба
    - Вопрос безопасности

    2. **Приоритет**:
    - Низкий (не срочно, косметические проблемы)
    - Средний (важно, но есть время)
    - Высокий (срочно, влияет на работу клиента)
    - Критический (немедленно, влияет на безопасность/бизнес/репутацию)

    3. **Тон клиента**:
    - Спокойный
    - Обеспокоенный
    - Недовольный
    - Агрессивный
    
    Структура ответа:
    - Номер тикета: #1001
    - Категория: технические проблемы
    - Приоритет: Средний
    - Тон клиента: Недовольный
    - Рекомендуемые действия: перезагрузите ПК

    ```{ticket}```'''

    response = clean_response(get_response(prompt))
    classifications.append(response)
    print(response)

print('=== СВОДКА ===')
prompt_classifications = f'''Резюмируй информацию по анализу тикетов обрамленной тройными обратными кавычками в ответе должны быть следующие параметры:
- Критических тикетов: колличество критических тикетов
- Высокий приоритет: колиичество тикетов с высоким приоритетом
- Требуют немедленного ответа: список номеров тикетов которые требуют немедленного ответа разделенные запятой

```{classifications}```'''

print(clean_response(get_response(prompt_classifications)))

=== КЛАССИФИКАЦИЯ ТИКЕТОВ ===
- Номер тикета: #1001  
- Категория: Технические проблемы  
- Приоритет: Высокий  
- Тон клиента: Обеспокоенный  
- Рекомендуемые действия: Проверьте работоспособность системы восстановления пароля, убедитесь в отправке подтверждения на электронные адреса и предложите клиенту альтернативный метод входа или инструкции по ручному восстановлению данных. Уведомите его о сроке обработки запроса, чтобы обеспечить оплату счета вовремя.
- Номер тикета: #1002  
- Категория: **Бизнес-предложение**  
- Приоритет: **Средний**  
- Тон клиента: **Спокойный**  
- Рекомендуемые действия: Перенаправить запрос в команду продаж/бизнес-развития для подготовки предложения по интеграции API и обсуждения корпоративных скидок. Предложить связаться напрямую с менеджером по клиенту.
- Номер тикета: #1003  
- Категория: Технические проблемы  
- Приоритет: Средний  
- Тон клиента: Спокойный  
- Рекомендуемые действия: Проверить версию приложения, убедиться, что оно обновлено, и прове

In [7]:
### Задача 3.2
booking_requests = [
    """Здравствуйте! Меня зовут Иван Петров, телефон +7-905-123-4567.
    Хочу забронировать конференц-зал "Альфа" на 15 марта с 10:00 до 16:00
    для проведения тренинга на 25 человек. Нужен проектор, флипчарт и
    кофе-брейк в 12:00. Email для подтверждения: i.petrov@company.ru""",

    """Добрый день! Мария Смирнова, +7-916-999-8888. Нужна переговорная
    комната на 20 февраля, время с 14:00 до 15:30. Участников будет 8 человек.
    Требуется телевизор с HDMI для презентации. Свяжитесь со мной по email
    m.smirnova@tech.com или в телеграм @msmirnova""",

    """Привет! Это Алексей Сидоров. Мой контакт: +7-903-555-7777. Хочу провести
    собеседования 28 февраля в вашем коворкинге. Понадобится небольшая комната
    на 4 человека с 9:00 до 18:00 (весь день). Желательно тихое место. Если есть
    вопросы - пишите на a.sidorov@startup.io"""
]

example_1 = """
Вход: "Добрый день, я Елена Волкова, +7-912-345-6789. Бронирую переговорную
на 10 марта с 15:00 до 17:00 на 6 человек. Email: e.volkova@biz.ru"

Выход:
{
  "client_name": "Елена Волкова",
  "phone": "+7-912-345-6789",
  "email": "e.volkova@biz.ru",
  "additional_contacts": null,
  "booking_date": "10 марта",
  "time_start": "15:00",
  "time_end": "17:00",
  "room_type": "переговорная",
  "room_name": null,
  "participants_count": 6,
  "additional_requirements": []
}
"""

example_2 = """
Вход: "Здравствуйте! Это Дмитрий Козлов. Мой номер: +7-926-777-4444,
почта d.kozlov@corp.com. Нужен зал "Бета" на 5 апреля с 9:00 до 13:00,
будет 15 человек. Нужны проектор и доска, плюс обед в 12:00."

Выход:
{
  "client_name": "Дмитрий Козлов",
  "phone": "+7-926-777-4444",
  "email": "d.kozlov@corp.com",
  "additional_contacts": null,
  "booking_date": "5 апреля",
  "time_start": "9:00",
  "time_end": "13:00",
  "room_type": "зал",
  "room_name": "Бета",
  "participants_count": 15,
  "additional_requirements": ["проектор", "доска", "обед в 12:00"]
}
"""

for i, request in enumerate(booking_requests):
    print(f'=== ЗАПРОС {i+1} ===')
    prompt = f'''Извлеки данные из запроса обрамленного тройными обратными кавычками. Структура ответа - JSON
    
    Примеры:
    {example_1}

    {example_2}

    ```{request}```'''

    print(clean_response(get_response(prompt)))

=== ЗАПРОС 1 ===
{
  "client_name": "Иван Петров",
  "phone": "+7-905-123-4567",
  "email": "i.petrov@company.ru",
  "additional_contacts": null,
  "booking_date": "15 марта",
  "time_start": "10:00",
  "time_end": "16:00",
  "room_type": "конференц-зал",
  "room_name": "Альфа",
  "participants_count": 25,
  "additional_requirements": ["проектор", "флипчарт", "кофе-брейк в 12:00"]
}
=== ЗАПРОС 2 ===
{
  "client_name": "Мария Смирнова",
  "phone": "+7-916-999-8888",
  "email": "m.smirnova@tech.com",
  "additional_contacts": ["@msmirnova"],
  "booking_date": "20 февраля",
  "time_start": "14:00",
  "time_end": "15:30",
  "room_type": "переговорная",
  "room_name": null,
  "participants_count": 8,
  "additional_requirements": ["телевизор с HDMI для презентации"]
}
=== ЗАПРОС 3 ===
{
  "client_name": "Алексей Сидоров",
  "phone": "+7-903-555-7777",
  "email": "a.sidorov@startup.io",
  "additional_contacts": null,
  "booking_date": "28 февраля",
  "time_start": "9:00",
  "time_end": "18:00"

In [8]:
## Часть 4
### Задача 4.1

sales_data = [
    {"product": "Ноутбук ASUS", "quantity": 5, "price": 45000},
    {"product": "Мышь Logitech", "quantity": 15, "price": 1500},
    {"product": "Клавиатура Keychron", "quantity": 8, "price": 7000},
    {"product": "Монитор Samsung", "quantity": 3, "price": 25000},
    {"product": "Наушники Sony", "quantity": 12, "price": 8000},
]

problem_description = f'''Создай python скрипт для анализа продаж интернет-магазина

ТРЕБОВАНИЯ:
1. **Функция `calculate_sales_metrics(sales_data)`**:
   - Принимает список словарей с информацией о продажах
   - Вычисляет: общую выручку, средний чек, топ-3 товара по выручке
   - Возвращает словарь с метриками

2. **Функция `generate_sales_report(metrics)`**:
   - Принимает словарь с метриками
   - Создает текстовый отчет в читаемом формате
   - Возвращает строку с отчетом

3. **Дополнительно**:
   - Валидация входных данных
   - Обработка ошибок (try-except)
   - Docstrings для всех функций
   - Пример использования в блоке `if __name__ == "__main__"`

**ПРИМЕРЫ ДАННЫХ:**
{sales_data}

ОЖИДАЕМЫЙ РЕЗУЛЬТАТ:
=== ОТЧЕТ О ПРОДАЖАХ ===
Общая выручка: 454,500 руб.
Количество заказов: 43
Средний чек: 10,569.77 руб.

Топ-3 товара по выручке:
1. Ноутбук ASUS - 225,000 руб.
2. Наушники Sony - 96,000 руб.
3. Монитор Samsung - 75,000 руб.
'''

print(clean_response(get_response(problem_description)))

```python
def calculate_sales_metrics(sales_data):
    """
    Рассчитывает ключевые метрики продаж на основе данных о продажах.

    Параметры:
        sales_data (list): Список словарей с информацией о каждой продаже.
                           Каждый словарь должен содержать 'product' (строка),
                           'quantity' (целое число или float), 'price' (целое число или float).

    Возвращает:
        dict: Словарь с метриками, включающий 'total_revenue' (общая выручка),
              'total_orders' (общее количество заказов), 'average_check' (средний чек),
              'top_products' (список из трёх кортежей (продукт, выручка)).

    Генерирует:
        ValueError: Если данные имеют неправильный формат или содержат недопустимые значения.
    """
    if not isinstance(sales_data, list):
        raise ValueError("sales_data должен быть списком")
    
    total_revenue = 0
    product_revenue = {}
    total_orders = 0

    for item in sales_data:
        if not isinstance

In [9]:
def calculate_sales_metrics(sales_data):
    """
    Calculate sales metrics from sales data.
    
    Args:
        sales_data (list of dicts): A list of dictionaries with sales data.
            Each dictionary should contain 'product', 'quantity', and 'price'.
    
    Returns:
        dict: A dictionary with metrics including total_revenue, total_orders, average_check, and top_products.
    
    Raises:
        ValueError: If the input data is invalid.
    """
    if not isinstance(sales_data, list):
        raise ValueError("Input data should be a list of dictionaries")
        
    total_revenue = 0
    total_orders = 0
    product_metrics = []
    
    for item in sales_data:
        if not isinstance(item, dict) or 'product' not in item or 'quantity' not in item or 'price' not in item:
            raise ValueError("Each item should be a dictionary with 'product', 'quantity', and 'price' keys")
            
        product = item['product']
        quantity = item['quantity']
        price = item['price']
        
        if not isinstance(quantity, int) or not isinstance(price, (int, float)):
            raise ValueError("Quantity and price should be numerical values")
        
        revenue = quantity * price
        total_revenue += revenue
        total_orders += quantity
        product_metrics.append({"product": product, "revenue": revenue})
    
    # Sort products by revenue in descending order
    product_metrics.sort(key=lambda x: x["revenue"], reverse=True)
    
    # Get top 3 products
    top_products = product_metrics[:3]
    
    average_check = total_revenue / total_orders if total_orders > 0 else 0
    
    return {
        "total_revenue": total_revenue,
        "total_orders": total_orders,
        "average_check": average_check,
        "top_products": top_products
    }


def generate_sales_report(metrics):
    """
    Generate a formatted sales report from sales metrics.
    
    Args:
        metrics (dict): A dictionary with sales metrics including total_revenue, total_orders, average_check, and top_products.
    
    Returns:
        str: A formatted sales report.
    """
    report = "=== ОТЧЕТ О ПРОДАЖАХ ===\n"
    report += f"Общая выручка: {metrics['total_revenue']:,.0f} руб.\n"
    report += f"Количество заказов: {metrics['total_orders']}\n"
    report += f"Средний чек: {metrics['average_check']:.2f} руб.\n\n"
    
    report += "Топ-3 товара по выручке:\n"
    for i, product in enumerate(metrics["top_products"], start=1):
        report += f"{i}. {product['product']} - {product['revenue']:,.0f} руб.\n"
        
    return report


if __name__ == "__main__":
    sample_data = [
        {'product': 'Ноутбук ASUS', 'quantity': 5, 'price': 45000},
        {'product': 'Мышь Logitech', 'quantity': 15, 'price': 1500},
        {'product': 'Клавиатура Keychron', 'quantity': 8, 'price': 7000},
        {'product': 'Монитор Samsung', 'quantity': 3, 'price': 25000},
        {'product': 'Наушники Sony', 'quantity': 12, 'price': 8000}
    ]
    
    try:
        metrics = calculate_sales_metrics(sample_data)
        report = generate_sales_report(metrics)
        print(report)
    except ValueError as e:
        print(f"Error: {e}")

=== ОТЧЕТ О ПРОДАЖАХ ===
Общая выручка: 474,500 руб.
Количество заказов: 43
Средний чек: 11034.88 руб.

Топ-3 товара по выручке:
1. Ноутбук ASUS - 225,000 руб.
2. Наушники Sony - 96,000 руб.
3. Монитор Samsung - 75,000 руб.



In [10]:
### Задача 4.2
code = '''
def process_orders(orders, min_amount=0):
    result = {}
    for order in orders:
        customer = order.get('customer')
        amount = order.get('amount', 0)
        if amount >= min_amount:
            if customer in result:
                result[customer] = result[customer] + amount
            else:
                result[customer] = amount
    return sorted(result.items(), key=lambda x: x[1], reverse=True)

orders_list = [
    {'customer': 'Alice', 'amount': 150},
    {'customer': 'Bob', 'amount': 200},
    {'customer': 'Alice', 'amount': 300},
    {'customer': 'Charlie', 'amount': 50},
    {'customer': 'Bob', 'amount': 100},
]

top_customers = process_orders(orders_list, min_amount=100)
print(top_customers)
'''

print('=== ШАГ 1: ОБЪЯСНЕНИЕ КОДА ===')
prompt_1 = f'''Подробно объясни работу кода обрамленного в тройные обратные кавычки. Сфокусируйся на следующих вопросах:
- Что делает функция?
- Как работает каждая строка?
- Что делает lambda в sorted()?
- Пример работы на конкретных данных

```{code}```
'''
print(clean_response(get_response(prompt_1)))
print('=== ШАГ 2: МОДИФИЦИРОВАННЫЙ КОД ===')
prompt_2 = f'''Модифицируй код обрамленный тройными обратными кавычками по следующим шагам:
- Добавь docstring
- Добавьте type hints
- Добавьте валидацию входных данных
- Добавьте обработку ошибок
- Переименуйте переменные для лучшей читаемости

```{code}```'''
new_code = clean_response(get_response(prompt_2))
print(new_code)
print('=== ШАГ 3: РАСШИРЕННАЯ ВЕРСИЯ ===')
prompt_3 = f'''Расширь функциональность кода обрамленного тройными обратными кавычками. Добавь следующий функционал:
- Добавьте параметр `top_n` для ограничения количества результатов
- Добавьте возможность сортировки по количеству заказов (не только по сумме)
- Верните результат в виде списка словарей вместо списка кортежей
```{new_code}```'''

print(clean_response(get_response(prompt_3)))

=== ШАГ 1: ОБЪЯСНЕНИЕ КОДА ===
**Объяснение работы кода**

**1. Что делает функция `process_orders`?**  
Функция фильтрует заказы по минимальной сумме, сгруппированные по клиентам, и возвращает список клиентов с их итоговыми суммами, отсортированный по убыванию.

---

**2. Пошаговое объяснение кода:**

```python
def process_orders(orders, min_amount=0):
```
- **`orders`** — список словарей с заказами.
- **`min_amount`** — минимальная сумма заказа (по умолчанию 0).

```python
    result = {}
```
- Создаётся пустой словарь `result` для хранения итоговых сумм по клиентам.

```python
    for order in orders:
```
- Цикл перебирает каждый заказ в `orders`.

```python
        customer = order.get('customer')
        amount = order.get('amount', 0)
```
- `customer` и `amount` извлекаются из текущего заказа. Если ключ `amount` отсутствует, используется значение по умолчанию `0`.

```python
        if amount >= min_amount:
```
- Проверяется, что сумма заказа `amount` не меньше `min_amount`.

```

In [11]:
## Часть 5
### Задача 5.1
course_reviews = [
    {
        "id": 1,
        "course": "Python для начинающих",
        "author": "Александр М.",
        "date": "2024-02-15",
        "text": "The course was absolutely amazing! The instructor explained complex topics "
                "like decorators and generators in a very simple way. However, I found "
                "the final project a bit too challenging and would have appreciated more "
                "guidance. Overall, highly recommend this course for intermediate learners.",
        "language": "en"
    },
    {
        "id": 2,
        "course": "Data Science Практикум",
        "author": "Мария К.",
        "date": "2024-02-18",
        "text": "Курс просто ужасный! Материалы устаревшие, видео записаны еще в 2020 году. "
                "Преподаватель монотонно читает слайды, никакой интерактивности. Задания "
                "не проверяются неделями. За такую цену ожидала намного большего. "
                "Требую возврата денег!",
        "language": "ru"
    },
    {
        "id": 3,
        "course": "Machine Learning Advanced",
        "author": "Дмитрий П.",
        "date": "2024-02-20",
        "text": "Отличный курс для тех, кто уже имеет базовые знания ML. Практические задания "
                "очень полезные, особенно понравился проект по созданию рекомендательной "
                "системы. Единственный минус - мало времени уделено нейронным сетям, "
                "хотелось бы больше углубиться в эту тему.",
        "language": "ru"
    }
]

token_cost = 0
input_token_price = 0.00 / 1000000
output_token_price = 0.00 / 1000000
input_token_amount = 0
output_token_amount = 0
responses = []

for i, review in enumerate(course_reviews):
    print('=========================')
    print(f'ОБРАБОТКА ОТЗЫВА №{i+1}')
    print('=========================')
    prompt = f'''
    Проанализируй отзыв обрамленный тройными обратными кавычками по следующим шагам:
    1. Если отзыв написан русском языке то переходи к следующему шагу, в противном случае переведи текст отзыва на русский язык
    2. Проанализируй настроение отзыва по шкале от 1 до 5 (1 - очень негативно, 5 - очень позитивно)
    3. Классифицируй отзыв по категории:
    - Восторженный отзыв
    - Положительный с замечаниями
    - Нейтральный
    - Негативный
    - Критический (требует немедленного реагирования)
    4. Извлеки ключевые моменты:
    - Что понравилось (плюсы)
    - Что не понравилось (минусы)
    - Предложения по улучшению (если есть)
    5. Сгенерируй ответ на отзыв от лица компании:
    - Учитывай тон отзыва
    - Адресуй конкретные пункты из отзыва
    - Если есть проблемы на которые жалуется пользователь предлагай решение
    - Поблагодари за обратную связь

    СТРУКТУРА ОТВЕТА:
    Курс: Python для начинающих
    Автор: Александр М.
    Дата: 2024-02-15

    --- ПЕРЕВОД НА РУССКИЙ ---
    [Переведенный текст]

    --- АНАЛИЗ НАСТРОЕНИЯ ---
    Оценка: 4/5 (Положительный)
    Категория: Положительный с замечаниями

    --- КЛЮЧЕВЫЕ МОМЕНТЫ ---
    Плюсы:
    - [пункт 1]
    - [пункт 2]

    Минусы:
    - [пункт 1]

    Предложения:
    - [пункт 1]

    --- СГЕНЕРИРОВАННЫЙ ОТВЕТ ---
    [Персонализированный ответ от компании]

    ```{review}```
    '''

    response = client.chat.completions.create(
        model=os.getenv('GROQ_MODEL'),
        messages=[{'role':'user', 'content':prompt}]
    )

    response_content = clean_response(response.choices[0].message.content)
    responses.append(response_content)
    print(response_content)
    input_token_amount += response.usage.prompt_tokens
    output_token_amount += response.usage.completion_tokens

prompt = f'''Проведи ревью анализа отзывов обрамленных в троные обратные кавычки, и выведи результат итоговой статистики.

ПРИМЕР ИТОГОВОЙ СТАТИСТИКИ:
=== ИТОГОВАЯ СТАТИСТИКА ===
Всего отзывов обработано: 3

Распределение по категориям:
- Восторженных: 0
- Положительных с замечаниями: 2
- Нейтральных: 0
- Негативных: 1
- Критических: 0

Средняя оценка: 3.7/5

```{responses}```
'''

print(clean_response(get_response(prompt)))
print(f'''Стоимость обратки:
- Входные токены: {input_token_amount}
- Выходные токены: {output_token_amount}
- Общая стоимость: ${input_token_amount * input_token_price + output_token_amount * output_token_price}''')


ОБРАБОТКА ОТЗЫВА №1
Курс: Python для начинающих  
Автор: Александр М.  
Дата: 2024-02-15  

--- ПЕРЕВОД НА РУССКИЙ ---  
Курс был абсолютно потрясающим! Инструктор объяснял сложные темы, такие как декораторы и генераторы, очень простым и понятным способом. Однако я нашёл финальный проект слишком сложным и по достоинству оценил бы дополнительные указания. В целом, однозначно рекомендую этот курс для среднего уровня.  

--- АНАЛИЗ НАСТРОЕНИЯ ---  
Оценка: 4/5 (Положительный)  
Категория: Положительный с замечаниями  

--- КЛЮЧЕВЫЕ МОМЕНТЫ ---  
Плюсы:  
- Простое и понятное объяснение сложных тем (декораторы, генераторы)  
- Высокая рекомендация для обучающихся на среднем уровне  

Минусы:  
- Сложность финального проекта и недостаток руководства  

Предложения:  
- Добавление подробных примеров или пошагового руководства для финального проекта  

--- СГЕНЕРИРОВАННЫЙ ОТВЕТ ---  
Уважаемый Александр,  

Спасибо за ваш отзыв! Мы рады, что вам понравилось объяснение сложных тем, а также рад